# 深度卷积神经网络（AlexNet）
:label:`sec_alexnet`

在LeNet提出后，卷积神经网络在计算机视觉和机器学习领域中很有名气。但卷积神经网络并没有主导这些领域。这是因为虽然LeNet在小数据集上取得了很好的效果，但是在更大、更真实的数据集上训练卷积神经网络的性能和可行性还有待研究。事实上，在上世纪90年代初到2012年之间的大部分时间里，神经网络往往被其他机器学习方法超越，如支持向量机（support vector machines）。

在计算机视觉中，直接将神经网络与其他机器学习方法进行比较也许不公平。这是因为，卷积神经网络的输入是由原始像素值或是经过简单预处理（例如居中、缩放）的像素值组成的。但在使用传统机器学习方法时，从业者永远不会将原始像素作为输入。在传统机器学习方法中，计算机视觉流水线是由经过人的手工精心设计的特征流水线组成的。对于这些传统方法，大部分的进展都来自于对特征有了更聪明的想法，并且学习到的算法往往归于事后的解释。

虽然上世纪90年代就有了一些神经网络加速卡，但仅靠它们还不足以开发出有大量参数的深层多通道多层卷积神经网络。此外，当时的数据集仍然相对较小。除了这些障碍，训练神经网络的一些关键技巧仍然缺失，包括启发式参数初始化、随机梯度下降的变体、非挤压激活函数和有效的正则化技术。

因此，与训练*端到端*（从像素到分类结果）系统不同，经典机器学习的流水线看起来更像下面这样：

1. 获取一个有趣的数据集。在早期，收集这些数据集需要昂贵的传感器（在当时最先进的图像也就100万像素）。
2. 根据光学、几何学、其他知识以及偶然的发现，手工对特征数据集进行预处理。
3. 通过标准的特征提取算法，如SIFT（尺度不变特征变换） :cite:`Lowe.2004`和SURF（加速鲁棒特征） :cite:`Bay.Tuytelaars.Van-Gool.2006`或其他手动调整的流水线来输入数据。
4. 将提取的特征送入最喜欢的分类器中（例如线性模型或其它核方法），以训练分类器。

当人们和机器学习研究人员交谈时，会发现机器学习研究人员相信机器学习既重要又美丽：优雅的理论去证明各种模型的性质。机器学习是一个正在蓬勃发展、严谨且非常有用的领域。然而，当人们和计算机视觉研究人员交谈，会听到一个完全不同的故事。计算机视觉研究人员会告诉一个诡异事实————推动领域进步的是数据特征，而不是学习算法。计算机视觉研究人员相信，从对最终模型精度的影响来说，更大或更干净的数据集、或是稍微改进的特征提取，比任何学习算法带来的进步要大得多。

## 学习表征

另一种预测这个领域发展的方法————观察图像特征的提取方法。在2012年前，图像特征都是机械地计算出来的。事实上，设计一套新的特征函数、改进结果，并撰写论文是盛极一时的潮流。SIFT :cite:`Lowe.2004`、SURF :cite:`Bay.Tuytelaars.Van-Gool.2006`、HOG（定向梯度直方图） :cite:`Dalal.Triggs.2005`、[bags of visual words](https://en.wikipedia.org/wiki/Bag-of-words_model_in_computer_vision)和类似的特征提取方法占据了主导地位。

另一组研究人员，包括Yann LeCun、Geoff Hinton、Yoshua Bengio、Andrew Ng、Shun ichi Amari和Juergen Schmidhuber，想法则与众不同：他们认为特征本身应该被学习。此外，他们还认为，在合理地复杂性前提下，特征应该由多个共同学习的神经网络层组成，每个层都有可学习的参数。在机器视觉中，最底层可能检测边缘、颜色和纹理。事实上，Alex Krizhevsky、Ilya Sutskever和Geoff Hinton提出了一种新的卷积神经网络变体*AlexNet*。在2012年ImageNet挑战赛中取得了轰动一时的成绩。AlexNet以Alex Krizhevsky的名字命名，他是论文 :cite:`Krizhevsky.Sutskever.Hinton.2012`的第一作者。

有趣的是，在网络的最底层，模型学习到了一些类似于传统滤波器的特征抽取器。 :numref:`fig_filters`是从AlexNet论文 :cite:`Krizhevsky.Sutskever.Hinton.2012`复制的，描述了底层图像特征。

![AlexNet第一层学习到的特征抽取器。](../img/filters.png)
:width:`400px`
:label:`fig_filters`

AlexNet的更高层建立在这些底层表示的基础上，以表示更大的特征，如眼睛、鼻子、草叶等等。而更高的层可以检测整个物体，如人、飞机、狗或飞盘。最终的隐藏神经元可以学习图像的综合表示，从而使属于不同类别的数据易于区分。尽管一直有一群执着的研究者不断钻研，试图学习视觉数据的逐级表征，然而很长一段时间里这些尝试都未有突破。深度卷积神经网络的突破出现在2012年。突破可归因于两个关键因素。

### 缺少的成分：数据

包含许多特征的深度模型需要大量的有标签数据，才能显著优于基于凸优化的传统方法（如线性方法和核方法）。
然而，限于早期计算机有限的存储和90年代有限的研究预算，大部分研究只基于小的公开数据集。例如，不少研究论文基于加州大学欧文分校（UCI）提供的若干个公开数据集，其中许多数据集只有几百至几千张在非自然环境下以低分辨率拍摄的图像。这一状况在2010年前后兴起的大数据浪潮中得到改善。2009年，ImageNet数据集发布，并发起ImageNet挑战赛：要求研究人员从100万个样本中训练模型，以区分1000个不同类别的对象。ImageNet数据集由斯坦福教授李飞飞小组的研究人员开发，利用谷歌图像搜索（Google Image Search）对每一类图像进行预筛选，并利用亚马逊众包（Amazon Mechanical Turk）来标注每张图片的相关类别。这种规模是前所未有的。这项被称为ImageNet的挑战赛推动了计算机视觉和机器学习研究的发展，挑战研究人员确定哪些模型能够在更大的数据规模下表现最好。

### 缺少的成分：硬件

深度学习对计算资源要求很高，训练可能需要数百个迭代轮数，每次迭代都需要通过代价高昂的许多线性代数层传递数据。这也是为什么在20世纪90年代至21世纪初，优化凸目标的简单算法是研究人员的首选。然而，用GPU训练神经网络改变了这一格局。*图形处理器*（Graphics Processing Unit，GPU）早年用来加速图形处理，使电脑游戏玩家受益。GPU可优化高吞吐量的$4 \times 4$矩阵和向量乘法，从而服务于基本的图形任务。幸运的是，这些数学运算与卷积层的计算惊人地相似。由此，英伟达（NVIDIA）和ATI已经开始为通用计算操作优化gpu，甚至把它们作为*通用GPU*（general-purpose GPUs，GPGPU）来销售。

那么GPU比CPU强在哪里呢？

首先，我们深度理解一下中央处理器（Central Processing Unit，CPU）的*核心*。
CPU的每个核心都拥有高时钟频率的运行能力，和高达数MB的三级缓存（L3Cache）。
它们非常适合执行各种指令，具有分支预测器、深层流水线和其他使CPU能够运行各种程序的功能。
然而，这种明显的优势也是它的致命弱点：通用核心的制造成本非常高。
它们需要大量的芯片面积、复杂的支持结构（内存接口、内核之间的缓存逻辑、高速互连等等），而且它们在任何单个任务上的性能都相对较差。
现代笔记本电脑最多有4核，即使是高端服务器也很少超过64核，因为它们的性价比不高。

相比于CPU，GPU由$100 \sim 1000$个小的处理单元组成（NVIDIA、ATI、ARM和其他芯片供应商之间的细节稍有不同），通常被分成更大的组（NVIDIA称之为warps）。
虽然每个GPU核心都相对较弱，有时甚至以低于1GHz的时钟频率运行，但庞大的核心数量使GPU比CPU快几个数量级。
例如，NVIDIA最近一代的Ampere GPU架构为每个芯片提供了高达312 TFlops的浮点性能，而CPU的浮点性能到目前为止还没有超过1 TFlops。
之所以有如此大的差距，原因其实很简单：首先，功耗往往会随时钟频率呈二次方增长。
对于一个CPU核心，假设它的运行速度比GPU快4倍，但可以使用16个GPU核代替，那么GPU的综合性能就是CPU的$16 \times 1/4 = 4$倍。
其次，GPU内核要简单得多，这使得它们更节能。
此外，深度学习中的许多操作需要相对较高的内存带宽，而GPU拥有10倍于CPU的带宽。

回到2012年的重大突破，当Alex Krizhevsky和Ilya Sutskever实现了可以在GPU硬件上运行的深度卷积神经网络时，一个重大突破出现了。他们意识到卷积神经网络中的计算瓶颈：卷积和矩阵乘法，都是可以在硬件上并行化的操作。
于是，他们使用两个显存为3GB的NVIDIA GTX580 GPU实现了快速卷积运算。他们的创新[cuda-convnet](https://code.google.com/archive/p/cuda-convnet/)几年来它一直是行业标准，并推动了深度学习热潮。

## AlexNet

2012年，AlexNet横空出世。它首次证明了学习到的特征可以超越手工设计的特征。它一举打破了计算机视觉研究的现状。
AlexNet使用了8层卷积神经网络，并以很大的优势赢得了2012年ImageNet图像识别挑战赛。

AlexNet和LeNet的架构非常相似，如 :numref:`fig_alexnet`所示。
注意，本书在这里提供的是一个稍微精简版本的AlexNet，去除了当年需要两个小型GPU同时运算的设计特点。

![从LeNet（左）到AlexNet（右）](../img/alexnet.svg)
:label:`fig_alexnet`

AlexNet和LeNet的设计理念非常相似，但也存在显著差异。

1. AlexNet比相对较小的LeNet5要深得多。AlexNet由八层组成：五个卷积层、两个全连接隐藏层和一个全连接输出层。
2. AlexNet使用ReLU而不是sigmoid作为其激活函数。

下面的内容将深入研究AlexNet的细节。

### 模型设计

在AlexNet的第一层，卷积窗口的形状是$11\times11$。
由于ImageNet中大多数图像的宽和高比MNIST图像的多10倍以上，因此，需要一个更大的卷积窗口来捕获目标。
第二层中的卷积窗口形状被缩减为$5\times5$，然后是$3\times3$。
此外，在第一层、第二层和第五层卷积层之后，加入窗口形状为$3\times3$、步幅为2的最大汇聚层。
而且，AlexNet的卷积通道数目是LeNet的10倍。

在最后一个卷积层后有两个全连接层，分别有4096个输出。
这两个巨大的全连接层拥有将近1GB的模型参数。
由于早期GPU显存有限，原版的AlexNet采用了双数据流设计，使得每个GPU只负责存储和计算模型的一半参数。
幸运的是，现在GPU显存相对充裕，所以现在很少需要跨GPU分解模型（因此，本书的AlexNet模型在这方面与原始论文稍有不同）。

### 激活函数

此外，AlexNet将sigmoid激活函数改为更简单的ReLU激活函数。
一方面，ReLU激活函数的计算更简单，它不需要如sigmoid激活函数那般复杂的求幂运算。
另一方面，当使用不同的参数初始化方法时，ReLU激活函数使训练模型更加容易。
当sigmoid激活函数的输出非常接近于0或1时，这些区域的梯度几乎为0，因此反向传播无法继续更新一些模型参数。
相反，ReLU激活函数在正区间的梯度总是1。
因此，如果模型参数没有正确初始化，sigmoid函数可能在正区间内得到几乎为0的梯度，从而使模型无法得到有效的训练。

### 容量控制和预处理

AlexNet通过暂退法（ :numref:`sec_dropout`）控制全连接层的模型复杂度，而LeNet只使用了权重衰减。
为了进一步扩充数据，AlexNet在训练时增加了大量的图像增强数据，如翻转、裁切和变色。
这使得模型更健壮，更大的样本量有效地减少了过拟合。
在 :numref:`sec_image_augmentation`中更详细地讨论数据扩增。


In [3]:
import torch
from torch import nn
from d2l import torch as d2l
import time
import numpy as np
import subprocess
import time
import pandas as pd

net = nn.Sequential(
    # 这里使用一个11*11的更大窗口来捕捉对象。
    # 同时，步幅为4，以减少输出的高度和宽度。
    # 另外，输出通道的数目远大于LeNet
    nn.Conv2d(1, 96, kernel_size=11, stride=4, padding=1), nn.ReLU(),
    nn.MaxPool2d(kernel_size=3, stride=2),
    # 减小卷积窗口，使用填充为2来使得输入与输出的高和宽一致，且增大输出通道数
    nn.Conv2d(96, 256, kernel_size=5, padding=2), nn.ReLU(),
    nn.MaxPool2d(kernel_size=3, stride=2),
    # 使用三个连续的卷积层和较小的卷积窗口。
    # 除了最后的卷积层，输出通道的数量进一步增加。
    # 在前两个卷积层之后，汇聚层不用于减少输入的高度和宽度
    nn.Conv2d(256, 384, kernel_size=3, padding=1), nn.ReLU(),
    nn.Conv2d(384, 384, kernel_size=3, padding=1), nn.ReLU(),
    nn.Conv2d(384, 256, kernel_size=3, padding=1), nn.ReLU(),
    nn.MaxPool2d(kernel_size=3, stride=2),
    nn.Flatten(),
    # 这里，全连接层的输出数量是LeNet中的好几倍。使用dropout层来减轻过拟合
    nn.Linear(6400, 4096), nn.ReLU(),
    nn.Dropout(p=0.5),
    nn.Linear(4096, 4096), nn.ReLU(),
    nn.Dropout(p=0.5),
    # 最后是输出层。由于这里使用Fashion-MNIST，所以用类别数为10，而非论文中的1000
    nn.Linear(4096, 10))

[**我们构造一个**]高度和宽度都为224的(**单通道数据，来观察每一层输出的形状**)。
它与 :numref:`fig_alexnet`中的AlexNet架构相匹配。


In [4]:
X = torch.randn(1, 1, 224, 224)
for layer in net:
    X=layer(X)
    print(layer.__class__.__name__,'output shape:\t',X.shape)

Conv2d output shape:	 torch.Size([1, 96, 54, 54])
ReLU output shape:	 torch.Size([1, 96, 54, 54])
MaxPool2d output shape:	 torch.Size([1, 96, 26, 26])
Conv2d output shape:	 torch.Size([1, 256, 26, 26])
ReLU output shape:	 torch.Size([1, 256, 26, 26])
MaxPool2d output shape:	 torch.Size([1, 256, 12, 12])
Conv2d output shape:	 torch.Size([1, 384, 12, 12])
ReLU output shape:	 torch.Size([1, 384, 12, 12])
Conv2d output shape:	 torch.Size([1, 384, 12, 12])
ReLU output shape:	 torch.Size([1, 384, 12, 12])
Conv2d output shape:	 torch.Size([1, 256, 12, 12])
ReLU output shape:	 torch.Size([1, 256, 12, 12])
MaxPool2d output shape:	 torch.Size([1, 256, 5, 5])
Flatten output shape:	 torch.Size([1, 6400])
Linear output shape:	 torch.Size([1, 4096])
ReLU output shape:	 torch.Size([1, 4096])
Dropout output shape:	 torch.Size([1, 4096])
Linear output shape:	 torch.Size([1, 4096])
ReLU output shape:	 torch.Size([1, 4096])
Dropout output shape:	 torch.Size([1, 4096])
Linear output shape:	 torch.Size([1,

## 读取数据集

尽管原文中AlexNet是在ImageNet上进行训练的，但本书在这里使用的是Fashion-MNIST数据集。因为即使在现代GPU上，训练ImageNet模型，同时使其收敛可能需要数小时或数天的时间。
将AlexNet直接应用于Fashion-MNIST的一个问题是，[**Fashion-MNIST图像的分辨率**]（$28 \times 28$像素）(**低于ImageNet图像。**)
为了解决这个问题，(**我们将它们增加到$224 \times 224$**)（通常来讲这不是一个明智的做法，但在这里这样做是为了有效使用AlexNet架构）。
这里需要使用`d2l.load_data_fashion_mnist`函数中的`resize`参数执行此调整。


In [5]:
batch_size = 128
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size, resize=224)
# print the shape of the train_iter
list_of_i = []
for i, (X, y) in enumerate(train_iter):
    list_of_i.append(i)

print('the shape of the train_iter is:', np.array(list_of_i).shape)
# print(list_of_i)
# print the first 10 batch of the train_iter
for i, (X, y) in enumerate(train_iter):
    if i < 10:
        print('the shape of the', i, 'batch of the train_iter is:', X.shape)
    else:
        break

100.0%


Extracting ../data/FashionMNIST/raw/train-images-idx3-ubyte.gz to ../data/FashionMNIST/raw



100.0%


Extracting ../data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ../data/FashionMNIST/raw



100.0%


Extracting ../data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ../data/FashionMNIST/raw



100.0%

Extracting ../data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/FashionMNIST/raw



the shape of the train_iter is: (469,)
the shape of the 0 batch of the train_iter is: torch.Size([128, 1, 224, 224])
the shape of the 1 batch of the train_iter is: torch.Size([128, 1, 224, 224])
the shape of the 2 batch of the train_iter is: torch.Size([128, 1, 224, 224])
the shape of the 3 batch of the train_iter is: torch.Size([128, 1, 224, 224])
the shape of the 4 batch of the train_iter is: torch.Size([128, 1, 224, 224])
the shape of the 5 batch of the train_iter is: torch.Size([128, 1, 224, 224])
the shape of the 6 batch of the train_iter is: torch.Size([128, 1, 224, 224])
the shape of the 7 batch of the train_iter is: torch.Size([128, 1, 224, 224])
the shape of the 8 batch of the train_iter is: torch.Size([128, 1, 224, 224])
the shape of the 9 batch of the train_iter is: torch.Size([128, 1, 224, 224])


## [**训练AlexNet**]

现在AlexNet可以开始被训练了。与 :numref:`sec_lenet`中的LeNet相比，这里的主要变化是使用更小的学习速率训练，这是因为网络更深更广、图像分辨率更高，训练卷积神经网络就更昂贵。


In [6]:
'''
对train_ch6()函数进行修改，使得能够在每一层的前向传播的时候，记录下时间, 
能耗部分不好进行具体统计，因为每个层的能耗有区别，并且时间过短，所以采用的方式为计算总能耗和总时长，最后通过平均值来计算估算的能耗
'''
def train_ch6self(net, train_iter, test_iter, num_epochs, lr, device):
    def init_weights(m): # 初始化权重
        if type(m) == nn.Linear or type(m) == nn.Conv2d:
            nn.init.xavier_uniform_(m.weight)
    net.apply(init_weights)
    # set a list of layer name
    list_layer_name = ['Conv2d','ReLU','MaxPool2d','Linear','Dropout','Flatten'] # 该模型中包括的所有的层的名字
    # create a numpy array to store the time and energy consumption, the shape is (num_epochs, len(list_layer_name), 2)
    # for each epoch, the shape is (6,2), contains the total time and the energy consumption of each layer
    timeenergy_data_forward = np.zeros((num_epochs, len(list_layer_name), 2)) 
    # for each epoch, the shape is (6,2), contains the total time of each part in a round, which is to_device, forward, loss, backward, optimizer，test_round
    # and the 2 means time and energy consumption, respectively
    timeenergy_data_round = np.zeros((num_epochs, 6, 2)) 
    # create another numpy array to store the data of each epoch, 1 column is training loss, 2 column is training accuracy, 3 column is test accuracy
    acc_data = []
    # create a numpy array to store the train loss and train accuracy
    train_l = []
    train_acc = []
    # create a numpy array to store the epoch and running time
    time_data_epoch = np.zeros((num_epochs, 2))
    # create a numpy array to store the energy consumption of each epoch
    energy_data_epoch = np.zeros((num_epochs, 1), dtype=object)
    # print the training device
    print('training on', device)
    net.to(device) # 将模型放到对应的设备上
    # 初始化optimizer和loss
    optimizer = torch.optim.SGD(net.parameters(), lr=lr)
    loss_fn = nn.CrossEntropyLoss()
    # 初始化计时器
    timer, num_batches = d2l.Timer(), len(train_iter)   
    # 开始训练
    for epoch in range(num_epochs):
        print('epoch %d' % (epoch + 1))
        # each epoch, set a timer to record the time
        timer.start()
        ##############################################################################################################
        # start the nvidia-smi command
        with open('gpu_power_usage.csv', 'w') as file:
            # Start the nvidia-smi command
            nvidia_smi_process = subprocess.Popen(
                ["nvidia-smi", "--query-gpu=power.draw", "--format=csv", "--loop-ms=1000"],
                stdout=file,  # Redirect the output directly to the file
                stderr=subprocess.PIPE,
                text=True
            )
        ##############################################################################################################
        net.train() # 设置为训练模式
        # 初始化每个epoch的统计时间的变量
        time_to_device_cost, time_forward, time_cost_loss = 0,0,0
        time_cost_backward, time_cost_optimizer, time_test_acc_cost = 0,0,0
        train_l_epoch = []
        train_acc_epoch = []
        metric = d2l.Accumulator(3)
        for i, (X, y) in enumerate(train_iter):
            print('round %d' % (i))
            time_round_start = time.time() # 计算每一轮的时间s
            optimizer.zero_grad() # 将optimizer的梯度清零s
            time_forward_epoch = np.zeros((len(list_layer_name), 1))
        ##################################################################################
            # 计算将数据放到对应的设备上的时间
            time_to_device_cost_i = 0
            time_to_device = time.time()
            X, y = X.to(device, non_blocking=True), y.to(device, non_blocking=True)
            time_to_device_end = time.time()
            time_to_device_cost_i = time_to_device_end - time_to_device
            print('time to device %f sec' % (time_to_device_cost_i))
            time_to_device_cost += time_to_device_cost_i
        ##################################################################################
            # 将原本的模型进行修改，使得能够逐层进行运行，并且在这个过程中，记录下时间和能量
            y_hat = X
            for layer in net:
                time_cost_layer = 0
                layer_name = layer.__class__.__name__ # 获取层的名字
                # find out the layer name is in where of the list
                layer_index = list_layer_name.index(layer_name)
                # calculate the time
                time_start_layer = time.time()
                y_hat = layer(y_hat)
                time_end_layer = time.time()
                time_cost_layer = time_end_layer - time_start_layer
                time_forward_epoch[layer_index,0] += time_cost_layer
                timeenergy_data_forward[epoch,layer_index,0] += time_cost_layer
                if torch.isinf(y_hat).any() or torch.isnan(y_hat).any():
                    print("Inf or NaN detected in y_hat")
            # 计算前向的时间
            time_forward = np.sum(timeenergy_data_forward[epoch,:,0])
            print('time forward %f sec' % (np.sum(time_forward_epoch)))
        ##################################################################################
            # 计算loss
            time_cost_loss_i = 0 # 初始化loss的时间
            time_start_loss = time.time()
            loss = loss_fn(y_hat, y)
            # print(loss)
            time_end_loss = time.time()
            time_cost_loss_i = time_end_loss - time_start_loss
            print('loss time %f sec' % (time_cost_loss_i))
            time_cost_loss += time_cost_loss_i
        ##################################################################################
            # 计算backward
            time_cost_backward_i = 0 # 初始化backward的时间
            time_start_backward = time.time()
            loss.backward()
            time_end_backward = time.time()
            time_cost_backward_i = time_end_backward - time_start_backward
            print('backward time %f sec' % (time_cost_backward_i))
            time_cost_backward += time_cost_backward_i
        ##################################################################################
            # 计算optimizer
            time_cost_optimizer_i = 0 # 初始化optimizer的时间
            time_start_optimizer = time.time()
            optimizer.step()
            time_end_optimizer = time.time()
            time_cost_optimizer_i = time_end_optimizer - time_start_optimizer
            print('optimizer time %f sec' % (time_cost_optimizer_i))
            time_cost_optimizer += time_cost_optimizer_i
        ##################################################################################
            time_round_end = time.time()
            time_round_cost = time_round_end - time_round_start
            print(f'training time in round {i} cost {time_round_cost} sec')
        ##################################################################################
            with torch.no_grad():
                metric.add(loss * X.shape[0], d2l.accuracy(y_hat, y), X.shape[0])
            train_l_i = metric[0] / metric[2]
            train_acc_i = metric[1] / metric[2]
            train_l_epoch.append(train_l_i)
            train_acc_epoch.append(train_acc_i)
            print('loss %f, train acc %f' % (train_l_i, train_acc_i))
        train_l.append(train_l_epoch)
        train_acc.append(train_acc_epoch)
        ##################################################################################
        # 进行模型的test部分运行
        time_test_acc_cost_epoch = 0
        time_test_acc_start = time.time()
        test_acc = d2l.evaluate_accuracy_gpu(net, test_iter, device)
        time_test_acc_end = time.time()
        time_test_acc_cost_epoch = time_test_acc_end - time_test_acc_start
        time_test_acc_cost += time_test_acc_cost_epoch
        print('test acc is %f' % (test_acc))
        acc_data.append(test_acc)
        ##################################################################################
        # 将每一轮的每个部分的时间加入到time_data_round中
        timeenergy_data_round[epoch,0,0] = time_to_device_cost
        timeenergy_data_round[epoch,1,0] = time_forward
        timeenergy_data_round[epoch,2,0] = time_cost_loss
        timeenergy_data_round[epoch,3,0] = time_cost_backward
        timeenergy_data_round[epoch,4,0] = time_cost_optimizer
        timeenergy_data_round[epoch,5,0] = time_test_acc_cost
        ##################################################################################
        # stop the nvidia-smi command
        nvidia_smi_process.terminate()
        timer.stop() # 停止计时 
        time_data_epoch[epoch,0] = epoch + 1
        time_data_epoch[epoch,1] = timer.sum()
        ##############################################################################################################
        # calculate the energy consumption of each epoch
        energy_datal = pd.read_csv('gpu_power_usage.csv')
        for l in range(len(energy_datal)):
            energy_datal.iloc[l,0] = energy_datal.iloc[l,0].replace(' W','')
        energy_datal = energy_datal.astype(float)  
        energy_datal = energy_datal.sum()
        energy_data_epoch[epoch,0] = energy_datal
        # print('epoch %d, time %f sec' % (epoch, timer.sum()))
    return timeenergy_data_forward, timeenergy_data_round, acc_data, train_l, train_acc, time_data_epoch, energy_data_epoch

In [7]:
lr, num_epochs = 0.01, 20
device = d2l.try_gpu()
timeenergy_data_forward, timeenergy_data_round, acc_data, train_l, train_acc, time_data_epoch, energy_data_epoch = train_ch6self(net, train_iter, test_iter, num_epochs, lr, device)

training on cuda:0
epoch 1
round 0
time to device 0.006511 sec
time forward 0.345123 sec
loss time 0.026628 sec
backward time 0.249378 sec
optimizer time 0.011804 sec
training time in round 0 cost 0.7027835845947266 sec
loss 2.305611, train acc 0.125000
round 1
time to device 0.004039 sec
time forward 0.001152 sec
loss time 0.000099 sec
backward time 0.001179 sec
optimizer time 0.000199 sec
training time in round 1 cost 0.04713749885559082 sec
loss 2.304097, train acc 0.117188
round 2
time to device 0.004511 sec
time forward 0.001469 sec
loss time 0.000085 sec
backward time 0.001223 sec
optimizer time 0.000245 sec
training time in round 2 cost 0.048090219497680664 sec
loss 2.304672, train acc 0.104167
round 3
time to device 0.004169 sec
time forward 0.001484 sec
loss time 0.000091 sec
backward time 0.001255 sec
optimizer time 0.000213 sec
training time in round 3 cost 0.04412221908569336 sec
loss 2.302614, train acc 0.111328
round 4
time to device 0.004061 sec
time forward 0.001191 sec

In [8]:
time_forward = timeenergy_data_forward[:,:,0]
time_forward

array([[0.59877086, 0.10500121, 0.05142021, 0.08697772, 0.04584932,
        0.00897074],
       [0.28587508, 0.09093213, 0.04961443, 0.07586479, 0.03646636,
        0.00904393],
       [0.29294276, 0.09231997, 0.05010462, 0.07688189, 0.03676295,
        0.0092752 ],
       [0.28234863, 0.08945847, 0.04914188, 0.07703257, 0.03675413,
        0.0091598 ],
       [0.28813958, 0.09076571, 0.04946661, 0.07612491, 0.03651619,
        0.00912929],
       [0.29876661, 0.09370923, 0.05117774, 0.07767105, 0.0370307 ,
        0.00932312],
       [0.2956171 , 0.09320521, 0.05064964, 0.07791877, 0.03714824,
        0.00918841],
       [0.28417635, 0.09004521, 0.04928398, 0.07726145, 0.03692031,
        0.00915027],
       [0.29492331, 0.09257579, 0.05044365, 0.07733059, 0.0368855 ,
        0.00922704],
       [0.29867864, 0.0956037 , 0.05135083, 0.07779336, 0.03739285,
        0.00931001],
       [0.29679179, 0.09423614, 0.05066895, 0.07665777, 0.03690076,
        0.00923777],
       [0.29281855, 0

In [9]:
time_round = timeenergy_data_round[:,:,0]
time_round

array([[1.89582515, 0.89699006, 0.07393169, 0.8366096 , 0.11032629,
        2.84555626],
       [1.89942431, 0.54779673, 0.05518675, 0.58306766, 0.09714484,
        2.87416577],
       [1.90760684, 0.55828738, 0.05451298, 0.60093498, 0.10437107,
        2.88428807],
       [1.90708113, 0.54389548, 0.04957867, 0.59477615, 0.10197997,
        2.87410426],
       [1.90321302, 0.55014229, 0.0509057 , 0.60187483, 0.10077024,
        2.91015196],
       [1.91447449, 0.56767845, 0.05369401, 0.60575104, 0.10498428,
        2.888592  ],
       [1.91307616, 0.56372738, 0.05298352, 0.60420418, 0.10338473,
        2.89238954],
       [1.90374565, 0.54683757, 0.04951191, 0.60788321, 0.10307193,
        2.89812613],
       [1.90572596, 0.56138587, 0.05364013, 0.60597825, 0.104774  ,
        2.88789153],
       [1.91457653, 0.57012939, 0.04994798, 0.60358262, 0.10379052,
        2.91053486],
       [1.91076541, 0.56449318, 0.05256701, 0.60598707, 0.10304236,
        2.89055467],
       [1.92141819, 0

In [10]:
test_acc = acc_data
test_acc

[0.7188,
 0.7845,
 0.8137,
 0.84,
 0.8486,
 0.8553,
 0.859,
 0.8722,
 0.8793,
 0.8786,
 0.8787,
 0.8808,
 0.8894,
 0.8828,
 0.8973,
 0.8948,
 0.8947,
 0.8992,
 0.9011,
 0.8884]

In [11]:
train_l

[[2.3056108951568604,
  2.3040966987609863,
  2.3046716849009194,
  2.3026143312454224,
  2.3026888370513916,
  2.302878220876058,
  2.302434512547084,
  2.3025638461112976,
  2.3020624849531384,
  2.3021290779113768,
  2.301771402359009,
  2.301379064718882,
  2.3011758144085226,
  2.30123393876212,
  2.3009531180063885,
  2.3007819205522537,
  2.300571287379545,
  2.3003641896777682,
  2.3007899836490027,
  2.3009241342544557,
  2.3005974406287786,
  2.3003357973965732,
  2.300154209136963,
  2.300407220919927,
  2.3003690910339354,
  2.3003312899516177,
  2.299904125708121,
  2.2996896505355835,
  2.2994924742600014,
  2.299337116877238,
  2.2991368539871706,
  2.2989013716578484,
  2.29886236335292,
  2.2986360718222225,
  2.2985228061676026,
  2.2983693612946405,
  2.2982520606066728,
  2.2981289123233997,
  2.2979410061469445,
  2.297804242372513,
  2.2975519168667677,
  2.2972997086388722,
  2.297116246334342,
  2.2971660603176463,
  2.2968134032355416,
  2.296666725822117,
  2.

In [12]:
train_acc

[[0.125,
  0.1171875,
  0.10416666666666667,
  0.111328125,
  0.1046875,
  0.10416666666666667,
  0.10825892857142858,
  0.107421875,
  0.10677083333333333,
  0.10625,
  0.10866477272727272,
  0.10807291666666667,
  0.10817307692307693,
  0.10602678571428571,
  0.1078125,
  0.10791015625,
  0.10983455882352941,
  0.11024305555555555,
  0.10896381578947369,
  0.109765625,
  0.11086309523809523,
  0.11221590909090909,
  0.11243206521739131,
  0.109375,
  0.11,
  0.1084735576923077,
  0.1099537037037037,
  0.10993303571428571,
  0.11018318965517242,
  0.11223958333333334,
  0.11164314516129033,
  0.11376953125,
  0.11410984848484848,
  0.11511948529411764,
  0.115625,
  0.11675347222222222,
  0.11739864864864864,
  0.11842105263157894,
  0.11979166666666667,
  0.1205078125,
  0.12157012195121951,
  0.12276785714285714,
  0.12318313953488372,
  0.12357954545454546,
  0.12552083333333333,
  0.12567934782608695,
  0.12483377659574468,
  0.12483723958333333,
  0.12531887755102042,
  0.125,
  

In [13]:
time_data_epoch

time_epoch = np.zeros((time_data_epoch.shape[0], time_data_epoch.shape[1]))
for i in range(len(time_data_epoch[:,0])):
    if i == 0:
        time_epoch[i,0] = time_data_epoch[i,0]
        time_epoch[i,1] = time_data_epoch[i,1]
    else:
        time_epoch[i,0] = time_data_epoch[i,0]
        time_epoch[i,1] = time_data_epoch[i,1] - time_data_epoch[i-1,1]
time_epoch

array([[ 1.        , 49.19019556],
       [ 2.        , 48.82736111],
       [ 3.        , 49.05926299],
       [ 4.        , 49.17564511],
       [ 5.        , 49.27827048],
       [ 6.        , 49.3755486 ],
       [ 7.        , 49.39823556],
       [ 8.        , 49.33143592],
       [ 9.        , 49.40453982],
       [10.        , 49.43203664],
       [11.        , 49.47626305],
       [12.        , 49.47726536],
       [13.        , 49.46333647],
       [14.        , 49.46429849],
       [15.        , 49.44848776],
       [16.        , 49.51685452],
       [17.        , 49.54309821],
       [18.        , 49.5515902 ],
       [19.        , 49.6170373 ],
       [20.        , 49.56146193]])

In [14]:
import os
working_dir = os.getcwd()
working_dir

# find out the parent directory
parent_dir1 = os.path.dirname(working_dir)
print(parent_dir1)

parent_dir2 = os.path.dirname(parent_dir1)
print(parent_dir2)

parent_dir3 = os.path.dirname(parent_dir2)
print(parent_dir3)

data_folder = os.path.join(parent_dir3, 'data')
print(data_folder)

/home/yj/FinalThesis/GreenAI/Alexnet_linux/code/GPU_nvidia-smi
/home/yj/FinalThesis/GreenAI/Alexnet_linux/code
/home/yj/FinalThesis/GreenAI/Alexnet_linux
/home/yj/FinalThesis/GreenAI/Alexnet_linux/data


In [15]:
# find the second_part folder
second_part_dir = os.path.join(data_folder, 'epoch_20SGD_GPU')
second_part_dir

'/home/yj/FinalThesis/GreenAI/Alexnet_linux/data/epoch_20SGD_GPU'

In [16]:
# save the data as .npy file
np.save(os.path.join(second_part_dir, 'time_forward.npy'), time_forward)
np.save(os.path.join(second_part_dir, 'time_round.npy'), time_round)
np.save(os.path.join(second_part_dir, 'test_acc.npy'), test_acc)
np.save(os.path.join(second_part_dir, 'train_acc.npy'), train_acc)
np.save(os.path.join(second_part_dir, 'train_l.npy'), train_l)
np.save(os.path.join(second_part_dir, 'time_epoch.npy'), time_epoch)
np.save(os.path.join(second_part_dir, 'energy_epoch.npy'), energy_data_epoch)